# Pretraining Training script in MAE

> Pretraining Training script in MAE

In [1]:
#| default_exp mae.pretraining_training

In [2]:
#| hide
%load_ext autoreload
%autoreload 2

In [3]:
#| export
import argparse
import datetime
import json
import numpy as np
from typing import Iterable
from types import SimpleNamespace
import os
import time
from pathlib import Path

import torch
import torch.backends.cudnn as cudnn
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import timm

2024-09-21 23:00:42.215491: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-21 23:00:42.604966: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-21 23:00:42.605021: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-21 23:00:42.605040: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-21 23:00:42.871513: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: S

In [4]:
timm.__version__

'1.0.9'

In [5]:
#| exporti
import segmentation_test.mae.misc as misc
import segmentation_test.mae.model_development as models_mae

In [6]:
#| export
import timm.optim.optim_factory as optim_factory
import math
from fastcore.test import *

In [7]:
torch.cuda.is_available()

True

### Gradient Scaling and Norm Counting: 


Dealing with mixed precision training, where we use both 16-bit and 32-bit floating-point numbers to speed up computation and reduce memory usage. But this can lead to some tricky numerical issues. Enter the `NativeScalerWithGradNormCount` class!

## What's this class all about?

This nifty little class is a wrapper around PyTorch's `GradScaler`. It's designed to handle the intricacies of mixed precision training while also giving us some extra goodies like gradient norm calculation.

## Let's break it down:

1. **Initialization**: We start by creating a `GradScaler` object. This is PyTorch's built-in tool for automatic mixed precision training.

2. **The `__call__` method**: This is where the magic happens!
   - It scales the loss and performs backpropagation.
   - If we're updating gradients, it handles gradient unscaling and clipping.
   - It also calculates the gradient norm, which is super useful for monitoring training stability.

3. **State management**: The `state_dict` and `load_state_dict` methods allow us to save and load the scaler's state. This is crucial for resuming training from checkpoints.

## Why is this so cool?

- It seamlessly integrates mixed precision training into our workflow.
- It provides gradient clipping out of the box, which helps prevent exploding gradients.
- The gradient norm calculation gives us valuable insights into our training process.

By using this class, we're not just training our model - we're training it smartly and efficiently. It's like having a personal trainer for your neural network!


In [8]:
#| export
class NativeScalerWithGradNormCount:
    state_dict_key = "amp_scaler"

    def __init__(self):
        self._scaler = torch.cuda.amp.GradScaler()

    def __call__(
		self, 
		loss, 
		optimizer, 
		clip_grad=None, 
		parameters=None, 
		create_graph=False, 
		update_grad=True):
        self._scaler.scale(loss).backward(create_graph=create_graph)
        if update_grad:
            if clip_grad is not None:
                assert parameters is not None
                self._scaler.unscale_(optimizer)  # unscale the gradients of optimizer's assigned params in-place
                norm = torch.nn.utils.clip_grad_norm_(parameters, clip_grad)
            else:
                self._scaler.unscale_(optimizer)
                norm = get_grad_norm_(parameters)
            self._scaler.step(optimizer)
            self._scaler.update()
        else:
            norm = None
        return norm

    def state_dict(self):
        return self._scaler.state_dict()

    def load_state_dict(self, state_dict):
        self._scaler.load_state_dict(state_dict)

In [9]:
#| export
def adjust_learning_rate(
	optimizer, 
	epoch, 
	args
	):
    """Decay the learning rate with half-cycle cosine after warmup"""
    if epoch < args.warmup_epochs:
        lr = args.lr * epoch / args.warmup_epochs 
    else:
        lr = args.min_lr + (args.lr - args.min_lr) * 0.5 * \
            (1. + math.cos(math.pi * (epoch - args.warmup_epochs) / (args.epochs - args.warmup_epochs)))
    for param_group in optimizer.param_groups:
        if "lr_scale" in param_group:
            param_group["lr"] = lr * param_group["lr_scale"]
        else:
            param_group["lr"] = lr
    return lr


In [10]:
#| export
def test_adjust_learning_rate():
    class DummyOptimizer:
        def __init__(self):
            self.param_groups = [
				{"lr": 0.1}, 
				{"lr": 0.2, "lr_scale": 2}
			]

    class DummyArgs:
        def __init__(self):
            self.warmup_epochs = 5
            self.epochs = 100
            self.lr = 0.1
            self.min_lr = 0.001

    optimizer = DummyOptimizer()
    args = DummyArgs()

    # Test during warmup
    lr = adjust_learning_rate(optimizer, 2, args)
    test_eq(lr, 0.04)
    test_eq(optimizer.param_groups[0]["lr"], 0.04)
    test_eq(optimizer.param_groups[1]["lr"], 0.08)

    # Test after warmup
    lr = adjust_learning_rate(optimizer, 50, args)
    expected_lr = 0.001 + (0.1 - 0.001) * 0.5 * (1 + math.cos(math.pi * 45 / 95))
    test_close(lr, expected_lr, eps=1e-6)
    test_close(optimizer.param_groups[0]["lr"], expected_lr, eps=1e-6)
    test_close(optimizer.param_groups[1]["lr"], expected_lr * 2, eps=1e-6)
    print("All tests passed!")


In [11]:
test_adjust_learning_rate()

All tests passed!


In [12]:
#| export
def train_one_epoch(
	            model: torch.nn.Module,
                data_loader: Iterable, 
				optimizer: torch.optim.Optimizer,
                device: torch.device, 
				epoch: int, 
				loss_scaler: NativeScalerWithGradNormCount,
                log_writer=None,
                args=None):
    model.train(True)
    metric_logger = misc.MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', misc.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)
    print_freq = 20

    accum_iter = args.accum_iter

    optimizer.zero_grad()

    if log_writer is not None:
        print('log_dir: {}'.format(log_writer.log_dir))

    for data_iter_step, (samples, _) in enumerate(metric_logger.log_every(data_loader, print_freq, header)):

        # we use a per iteration (instead of per epoch) lr scheduler
        if data_iter_step % accum_iter == 0:
            lr_sched.adjust_learning_rate(optimizer, data_iter_step / len(data_loader) + epoch, args)

        samples = samples.to(device, non_blocking=True)

        with torch.cuda.amp.autocast():
            loss, _, _ = model(samples, mask_ratio=args.mask_ratio)

        loss_value = loss.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            sys.exit(1)

        loss /= accum_iter
        loss_scaler(
			loss, 
			optimizer, 
			parameters=model.parameters(),
            update_grad=(data_iter_step + 1) % accum_iter == 0
		)
        if (data_iter_step + 1) % accum_iter == 0:
        	optimizer.zero_grad()
        torch.cuda.synchronize()

        metric_logger.update(loss=loss_value)

        lr = optimizer.param_groups[0]["lr"]
        metric_logger.update(lr=lr)

        loss_value_reduce = misc.all_reduce_mean(loss_value)
        if log_writer is not None and (data_iter_step + 1) % accum_iter == 0:
            """ We use epoch_1000x as the x-axis in tensorboard.
			This calibrates different curves when batch size changes.
			"""
            epoch_1000x = int((data_iter_step / len(data_loader) + epoch) * 1000)
            log_writer.add_scalar('train_loss', loss_value_reduce, epoch_1000x)
            log_writer.add_scalar('lr', lr, epoch_1000x)


	# gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    return {k: meter.global_avg for k, meter in metric_logger.meters.items()}

# main_pretrain.py script will be implemented here

> engine_pretrain.py needs to be implemented here, and call main function

In [13]:
from fastai.vision.all import untar_data, URLs

# Download and extract the ImageWang dataset (a subset of ImageNet)
path = untar_data(URLs.IMAGEWANG_160)

# Update the DATA_PATH to point to the downloaded dataset
DATA_PATH = str(path)

print(f"ImageWang dataset downloaded and extracted to: {DATA_PATH}")


ImageWang dataset downloaded and extracted to: /home/user/.fastai/data/imagewang-160


In [14]:
Path(DATA_PATH).ls()
# MAE pre-training arguments

(#3) [Path('/home/user/.fastai/data/imagewang-160/val'),Path('/home/user/.fastai/data/imagewang-160/train'),Path('/home/user/.fastai/data/imagewang-160/unsup')]

In [15]:
# MAE pre-training arguments
BATCH_SIZE = 64  # Batch size per GPU (effective batch size is BATCH_SIZE * ACCUM_ITER * # gpus)
EPOCHS = 400
ACCUM_ITER = 1  # Accumulate gradient iterations (for increasing the effective batch size under memory constraints)

# Model parameters
MODEL = 'mae_vit_large_patch16'  # Name of model to train
INPUT_SIZE = 224  # images input size
MASK_RATIO = 0.75  # Masking ratio (percentage of removed patches)
NORM_PIX_LOSS = False  # Use (per-patch) normalized pixels as targets for computing loss

# Optimizer parameters
WEIGHT_DECAY = 0.05
LR = None  # learning rate (absolute lr)
BLR = 1e-3  # base learning rate: absolute_lr = base_lr * total_batch_size / 256
MIN_LR = 0.  # lower lr bound for cyclic schedulers that hit 0
WARMUP_EPOCHS = 40  # epochs to warmup LR

# Dataset parameters
DATA_PATH = str(path)  # dataset path
# Typically, this folder structure contains:
# - 'train' folder: Contains subfolders for each class, with training images
# - 'val' folder: Contains subfolders for each class, with validation images
# - 'test' folder: (optional) Contains test images, if provided
# - 'labels.txt' or similar: A file mapping class names to numeric IDs
OUTPUT_DIR = './output_dir'  # path where to save, empty for no saving
LOG_DIR = './output_dir'  # path where to tensorboard log
DEVICE = 'cuda'  # device to use for training / testing
SEED = 0
RESUME = ''  # resume from checkpoint
START_EPOCH = 0
NUM_WORKERS = 4# 10 was too much for my GPU
PIN_MEM = True  # Pin CPU memory in DataLoader for more efficient (sometimes) transfer to GPU.

# distributed training parameters
WORLD_SIZE = 1  # number of distributed processes
LOCAL_RANK = -1
DIST_ON_ITP = False
DIST_URL = 'env://'  # url used to set up distributed training


In [16]:
# Create a SimpleNamespace object to mimic argparse.Namespace
args = SimpleNamespace(
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    accum_iter=ACCUM_ITER,
    model=MODEL,
    input_size=INPUT_SIZE,
    mask_ratio=MASK_RATIO,
    norm_pix_loss=NORM_PIX_LOSS,
    weight_decay=WEIGHT_DECAY,
    lr=LR,
    blr=BLR,
    min_lr=MIN_LR,
    warmup_epochs=WARMUP_EPOCHS,
    data_path=DATA_PATH,
    output_dir=OUTPUT_DIR,
    log_dir=LOG_DIR,
    device=DEVICE,
    seed=SEED,
    resume=RESUME,
    start_epoch=START_EPOCH,
    num_workers=NUM_WORKERS,
    pin_mem=PIN_MEM,
    world_size=WORLD_SIZE,
    local_rank=LOCAL_RANK,
    dist_on_itp=DIST_ON_ITP,
    dist_url=DIST_URL
)


In [17]:
misc.init_distributed_mode(args)

Not using distributed mode


In [18]:
print('job dir: {}'.format(os.getcwd()))

[23:01:06.230455] job dir: /home/user/Schreibtisch/projects/git_data/segmentation_test/nbs


In [19]:
device = torch.device(args.device)
device

device(type='cuda')

In [20]:
# fix the seed for reproducibility
seed = args.seed + misc.get_rank()
torch.manual_seed(seed)
np.random.seed(seed)
cudnn.benchmark = True

In [21]:
# simple augmentation
transform_train = transforms.Compose([
            transforms.RandomResizedCrop(
                args.input_size, 
                scale=(0.2, 1.0), 
                interpolation=3),  # 3 is bicubic
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225])])

In [22]:
transform_train

Compose(
    RandomResizedCrop(size=(224, 224), scale=(0.2, 1.0), ratio=(0.75, 1.3333), interpolation=bicubic, antialias=True)
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

In [23]:
dataset_train = datasets.ImageFolder(
    os.path.join(
        args.data_path, 'train'), 
        transform=transform_train)

In [24]:
dataset_train

Dataset ImageFolder
    Number of datapoints: 14669
    Root location: /home/user/.fastai/data/imagewang-160/train
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(224, 224), scale=(0.2, 1.0), ratio=(0.75, 1.3333), interpolation=bicubic, antialias=True)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [25]:
if True:  # args.distributed:
    num_tasks = misc.get_world_size()
    global_rank = misc.get_rank()
    sampler_train = torch.utils.data.DistributedSampler(
        dataset_train, num_replicas=num_tasks, rank=global_rank, shuffle=True
    )
    print("Sampler_train = %s" % str(sampler_train))
else:
    sampler_train = torch.utils.data.RandomSampler(dataset_train)

[23:01:11.562099] Sampler_train = <torch.utils.data.distributed.DistributedSampler object at 0x76348e6be190>


In [26]:
if global_rank == 0 and args.log_dir is not None:
    os.makedirs(args.log_dir, exist_ok=True)
    log_writer = SummaryWriter(log_dir=args.log_dir)
else:
    log_writer = None

In [27]:
data_loader_train = torch.utils.data.DataLoader(
        dataset_train, sampler=sampler_train,
        batch_size=args.batch_size,
        num_workers=args.num_workers,
        pin_memory=args.pin_mem,
        drop_last=True,
    )

In [28]:
args.model

'mae_vit_large_patch16'

In [29]:

# define the model
model = models_mae.__dict__[args.model](norm_pix_loss=args.norm_pix_loss)
model

MaskedAutoencoderViT(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-23): 24 x Block(
      (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1024, out_features=3072, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
      

In [30]:
model.to(device)

model_without_ddp = model

In [ ]:

def main(args):
    # Your main function implementation here
    print("Starting MAE pre-training with the following arguments:")
    for arg, value in vars(args).items():
        print(f"{arg}: {value}")
    
    # Add your training logic here
    # For example:
    # model = create_model(args)
    # dataset = load_dataset(args)
    # train_one_epoch(model, dataset, args)
    
    print("MAE pre-training completed.")

# To run the main function, you can now use:
# main(args)
